algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.
* a capacitação dos profissionais de saúde influencia no tamanho das filas.

In [ ]:
class Fila():
    def __init__(self):
        self.data = []

    def inserir(self, x):
        self.data.append(x)

    def remover(self):
        if len(self.data) > 0:
            return self.data.pop(0)

    def top(self):
        if len(self.data) > 0:
            return self.data[0]

    def empty(self):
        return not len(self.data) > 0

    def exibir(self):
        return list(self.data)